## IMAGE CLASSIFICATION FOR FOOD RECOGNITION

### Loading Dataset

#### Uploading keegle.json File

In [1]:
from google.colab import files
files.upload()  # Upload the kaggle.json file here


Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"rishikkurkiya","key":"3f23546c1419fc3ea7541b73d5fa9664"}'}

#### Importing Dataset from keggle

In [2]:
import os
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json


In [3]:
!kaggle datasets download -d kmader/food41
!unzip -q food41.zip -d food41


Dataset URL: https://www.kaggle.com/datasets/kmader/food41
License(s): copyright-authors
100% 5.29G/5.30G [01:08<00:00, 98.1MB/s]
100% 5.30G/5.30G [01:08<00:00, 83.3MB/s]


### Process The Data

In [10]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np
import os
import shutil
import random

base_dir = 'food41/images'
subset_dir = 'food41_subset'

# Define subset size
subset_size = 5000  # Total number of images to use for training and validation

# Create a subset directory
if not os.path.exists(subset_dir):
    os.makedirs(subset_dir)
    categories = os.listdir(base_dir)
    for category in categories:
        category_dir = os.path.join(base_dir, category)
        subset_category_dir = os.path.join(subset_dir, category)
        os.makedirs(subset_category_dir, exist_ok=True)
        images = os.listdir(category_dir)
        subset_images = random.sample(images, min(len(images), subset_size // len(categories)))
        for image in subset_images:
            shutil.copy(os.path.join(category_dir, image), os.path.join(subset_category_dir, image))

# Data augmentation and normalization for training
train_datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2,  # split dataset into training and validation
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)

train_generator = train_datagen.flow_from_directory(
    subset_dir,
    target_size=(150, 150),
    batch_size=32,
    class_mode='categorical',
    subset='training'  # set as training data
)

validation_generator = train_datagen.flow_from_directory(
    subset_dir,
    target_size=(150, 150),
    batch_size=32,
    class_mode='categorical',
    subset='validation'  # set as validation data
)


Found 4040 images belonging to 101 classes.
Found 909 images belonging to 101 classes.


### Building The Model

In [11]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(150, 150, 3)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(512, activation='relu'),
    Dropout(0.5),
    Dense(101, activation='softmax')
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()


Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_6 (Conv2D)           (None, 148, 148, 32)      896       
                                                                 
 max_pooling2d_6 (MaxPoolin  (None, 74, 74, 32)        0         
 g2D)                                                            
                                                                 
 conv2d_7 (Conv2D)           (None, 72, 72, 64)        18496     
                                                                 
 max_pooling2d_7 (MaxPoolin  (None, 36, 36, 64)        0         
 g2D)                                                            
                                                                 
 conv2d_8 (Conv2D)           (None, 34, 34, 128)       73856     
                                                                 
 max_pooling2d_8 (MaxPoolin  (None, 17, 17, 128)      

### Training and Evaluating the Model

#### Training the model

In [13]:
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // validation_generator.batch_size,
    epochs=10
)


Epoch 1/10
126/126 [==============================] - 249s 2s/step - loss: 4.6164 - accuracy: 0.0095 - val_loss: 4.5923 - val_accuracy: 0.0156
Epoch 2/10
126/126 [==============================] - 241s 2s/step - loss: 4.5672 - accuracy: 0.0165 - val_loss: 4.5383 - val_accuracy: 0.0134
Epoch 3/10
126/126 [==============================] - 244s 2s/step - loss: 4.5077 - accuracy: 0.0297 - val_loss: 4.5259 - val_accuracy: 0.0279
Epoch 4/10
126/126 [==============================] - 248s 2s/step - loss: 4.4371 - accuracy: 0.0374 - val_loss: 4.4352 - val_accuracy: 0.0368
Epoch 5/10
126/126 [==============================] - 245s 2s/step - loss: 4.3377 - accuracy: 0.0529 - val_loss: 4.4025 - val_accuracy: 0.0312
Epoch 6/10
126/126 [==============================] - 251s 2s/step - loss: 4.2265 - accuracy: 0.0639 - val_loss: 4.3882 - val_accuracy: 0.0458
Epoch 7/10
126/126 [==============================] - 244s 2s/step - loss: 4.0862 - accuracy: 0.0898 - val_loss: 4.3766 - val_accuracy: 0.0513

#### Evaluating The Model

In [14]:
loss, accuracy = model.evaluate(validation_generator, steps=validation_generator.samples // validation_generator.batch_size)
print(f'Validation accuracy: {accuracy*100:.2f}%')


28/28 [==============================] - 17s 596ms/step - loss: 4.4686 - accuracy: 0.0536
Validation accuracy: 5.36%


### Making Prediction

In [16]:
import urllib.request

# URL of the sample image
image_url = 'https://images.unsplash.com/photo-1600891964599-f61ba0e24092?ixlib=rb-1.2.1&ixid=MnwxMjA3fDB8MHxwaG90by1wYWdlfHx8fGVufDB8fHx8&auto=format&fit=crop&w=800&q=80'

# Download the image
image_path = 'sample_food_image.jpg'
urllib.request.urlretrieve(image_url, image_path)

print(f"Image downloaded to {image_path}")


Image downloaded to sample_food_image.jpg


In [19]:
import numpy as np
from tensorflow.keras.preprocessing import image

def predict_image(img_path):
    img = image.load_img(img_path, target_size=(150, 150))
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0) / 255.0
    prediction = model.predict(img_array)
    return np.argmax(prediction)

# Example prediction
img_path = '/content/sample_food_image.jpg'
predicted_class = predict_image(img_path)
print(f'Predicted class: {predicted_class}')


1/1 [==============================] - 0s 227ms/step
Predicted class: 95
